# Session 3: 예측 모델 개발 및 학습 (Model Development & Training)

### ■ Session Objective
- Vertex AI 개발 환경에서 금융 데이터를 기반으로 대출 승인 여부를 예측하는 머신러닝 모델을 개발하고, 학습된 모델 아티팩트(Artifact)를 생성함.

### ■ Key Process
1.  **Data Ingestion (데이터 로드)**
    - Cloud Storage(GCS)에 적재된 원본 데이터셋(`loan_data.csv`)을 Pandas DataFrame으로 로드.

2.  **Data Preprocessing (데이터 전처리)**
    - 결측치 처리, 범주형 변수 인코딩 등 모델 학습에 적합한 형태로 데이터를 정제 및 변환.
    - 모델의 입력 변수(Features)와 목표 변수(Target)를 정의하고 분리.

3.  **Model Training (모델 학습)**
    - Scikit-learn 라이브러리의 `RandomForestClassifier` 알고리즘을 사용하여 분류 모델을 학습.
    - 학습용(Train) 데이터와 검증용(Test) 데이터를 분리하여 모델 성능을 객관적으로 평가.

4.  **Model Serialization (모델 직렬화)**
    - 학습 완료된 모델 객체(Trained Model)를 추후 배포 및 재사용이 가능한 파일(`.pkl`) 형태로 직렬화하여 저장.

### ■ Deliverable
- `model.pkl`: 배포 및 서빙을 위해 준비된 직렬화된 모델 아티팩트.
- `model_features.pkl`: 추론 시점의 데이터 일관성을 보장하기 위한 피처 목록 파일.# Session 3: 예측 모델 개발 및 학습 (Model Development & Training)

### ■ Session Objective
- Vertex AI 개발 환경에서 금융 데이터를 기반으로 대출 승인 여부를 예측하는 머신러닝 모델을 개발하고, 학습된 모델 아티팩트(Artifact)를 생성함.

### ■ Key Process
1.  **Data Ingestion (데이터 로드)**
    - Cloud Storage(GCS)에 적재된 원본 데이터셋(`train_loan_prediction.csv`)을 Pandas DataFrame으로 로드.

2.  **Data Preprocessing (데이터 전처리)**
    - 결측치 처리, 범주형 변수 인코딩 등 모델 학습에 적합한 형태로 데이터를 정제 및 변환.
    - 모델의 입력 변수(Features)와 목표 변수(Target)를 정의하고 분리.

3.  **Model Training (모델 학습)**
    - Scikit-learn 라이브러리의 `RandomForestClassifier` 알고리즘을 사용하여 분류 모델을 학습.
    - 학습용(Train) 데이터와 검증용(Test) 데이터를 분리하여 모델 성능을 객관적으로 평가.

4.  **Model Serialization (모델 직렬화)**
    - 학습 완료된 모델 객체(Trained Model)를 추후 배포 및 재사용이 가능한 파일(`.pkl`) 형태로 직렬화하여 저장.

### ■ Deliverable
- `model.pkl`: 배포 및 서빙을 위해 준비된 직렬화된 모델 아티팩트.
- `model_features.pkl`: 추론 시점의 데이터 일관성을 보장하기 위한 피처 목록 파일.

## 0. 사전 준비 (Prerequisites)

원활한 실습 진행을 위해, 교육 시작 전 아래 두 가지 준비 사항을 반드시 완료해주시기 바랍니다.

---

### 1. 실습 데이터셋 다운로드

-   **데이터셋 출처:** [Loan Prediction Problem Dataset](https://www.kaggle.com/datasets/altruistdelhite04/loan-prediction-problem-dataset)
-   **Github:** [세션3-데이터셋](https://github.com/YOONSEOKHEO/AgenticAI-HandsOn/blob/main/1_VertexAI/Session3/loan_data/train_loan_prediction.csv)
-   **필수 파일:** `train_loan_prediction.csv`
    -   *상기 링크에 접속하여 `train_loan_prediction.csv` 파일을 로컬 PC에 다운로드합니다.*  
    -   오른쪽 상단 다운로드 버튼(Download_raw_file)
    -   만약 git clone을 로컬에 해두었다면 다운로드 할 필요 없음

---

### 2. Google Cloud Storage(GCS) 버킷 생성

-   **목적:** 다운로드한 데이터셋을 업로드하고, 향후 생성될 모델 아티팩트를 저장하기 위한 클라우드 스토리지 공간을 확보합니다.

-   **생성 절차:**
    1.  GCP Console 접속 후, 상단 검색창을 통해 'Cloud Storage' 서비스로 이동합니다.
    2. 왼쪽 바에 **2.버킷
    2.  `+ 만들기` 버튼을 클릭하여 버킷 생성 프로세스를 시작합니다.
    3.  아래 가이드에 따라 버킷 속성을 설정합니다.
        -   **이름 (Name):** 전역적으로 고유한(Globally unique) 이름으로 설정
            -   *권장 형식: `[GCP-Project-ID]-[이름이니셜]-train-loan-data`*
            -   *예시: `my_instance-ysh-train-loan-data`*
        -   **위치 유형 (Location type):** `Region`
        -   **리전 (Region):** `asia-northeast3 (Seoul)`
        -   *나머지 설정은 모두 기본값을 유지합니다.*
    4.  `만들기` 버튼을 클릭하여 버킷 생성을 완료합니다.
    5.  `업로드 버튼을 통해 대출 심사 데이터 업로드 하기 

In [1]:
print("Hello world")

Hello world


In [2]:
# ==============================================================================
# Session 3: AI 대출 심사관 모델 학습 (Full Code)
# ==============================================================================

# ==============================================================================
# 단계 1: 라이브러리 가져오기 및 환경 설정
# ==============================================================================
# 데이터 분석을 위한 pandas, 모델 저장을 위한 joblib 등을 가져옵니다.
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib, pickle
import os
import sys # 시스템 관련 함수를 사용하기 위해 임포트

print("Step 1: 필요한 라이브러리를 모두 가져왔습니다.")

# --- scikit-learn 버전 검사 및 조정 ---
REQUIRED_SKLEARN_VERSION = "1.5.2" # Vertex AI Pre-built Container와 호환되는 버전

current_sklearn_version = sklearn.__version__
print(f"현재 scikit-learn 버전: {current_sklearn_version}")

if current_sklearn_version != REQUIRED_SKLEARN_VERSION:
    print(f"경고: 필요한 scikit-learn 버전({REQUIRED_SKLEARN_VERSION})과 다릅니다.")
    print(f"     자동으로 '{REQUIRED_SKLEARN_VERSION}' 버전으로 다운그레이드/설치를 진행합니다.")
    
    # pip를 사용하여 지정된 버전 설치
    !pip install scikit-learn=={REQUIRED_SKLEARN_VERSION} --quiet
    
    # 설치 후, Jupyter 커널을 다시 시작하도록 안내 (변경 사항 적용을 위함)
    print(f"\n====================================================================================")
    print(f"▶ scikit-learn {REQUIRED_SKLEARN_VERSION} 설치 완료! ")
    print(f"▶ 변경 사항 적용을 위해 JupyterLab 'Kernel' -> 'Restart Kernel...'을 클릭하여")
    print(f"▶ 커널을 반드시 '다시 시작' 해주세요. 그 후에 다시 이 셀부터 실행하시면 됩니다.")
    print(f"====================================================================================")
    
    # 커널 재시작 메시지를 출력하고 스크립트 실행을 중단하여 사용자가 재시작하도록 유도
    # (실제 환경에서는 sys.exit()를 사용할 수 있지만, Jupyter에서는 권장하지 않습니다.)
    # 여기서는 안내 메시지를 출력하는 것으로 충분합니다.
    # sys.exit() # JupyterLab에서 sys.exit()는 셀 실행만 중단하고 커널을 죽이지 않습니다.

# --- (선택 사항: 재시작 후 버전 재확인) ---
# 커널 재시작 후 이 셀을 다시 실행하면, 아래 라인에서
# "scikit-learn 버전이 일치합니다." 메시지를 볼 수 있습니다.
# import sklearn # 재시작 후에는 다시 임포트해야 최신 버전 반영
# if sklearn.__version__ == REQUIRED_SKLEARN_VERSION:
#     print(f"scikit-learn 버전이 {REQUIRED_SKLEARN_VERSION}으로 일치합니다.")
# else:
#     print(f"오류: scikit-learn 버전이 여전히 일치하지 않습니다. 수동 재시작을 확인해주세요.")


Step 1: 필요한 라이브러리를 모두 가져왔습니다.
현재 scikit-learn 버전: 1.5.2


In [3]:
# ------------------------------------------------------------------------------
# 단계 2: GCS에서 데이터 불러오기
# ------------------------------------------------------------------------------
# Google Cloud Storage 버킷에 있는 데이터를 직접 읽어옵니다.
# 'gs://' 프로토콜을 사용하면 됩니다.

# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ 수강생 각자의 버킷 이름으로 이 부분을 수정해야 합니다! ※
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※

BUCKET_NAME = "my_instance-ysh2-banking-data"  # 예: "gcp-project-12345-loan-data"
TRAIN_FILE = "train_loan_prediction.csv"
GCS_PATH = f"gs://{BUCKET_NAME}/{TRAIN_FILE}"

try:
    df = pd.read_csv(GCS_PATH)
    print("\nStep 2: GCS에서 데이터를 성공적으로 불러왔습니다.")
    print(f"  - 원본 데이터 (Train) : {df.shape[0]} 건")
    print("\n  - 원본 데이터 미리보기:")
    print(df.head())
except Exception as e:
    print(f"\nStep 2: 데이터 로드 중 오류 발생! 버킷 이름과 파일 이름을 확인하세요: {e}")




Step 2: GCS에서 데이터를 성공적으로 불러왔습니다.
  - 원본 데이터 (Train) : 614 건

  - 원본 데이터 미리보기:
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban  

In [4]:
# ------------------------------------------------------------------------------
# 단계 3: 데이터 전처리 (Data Preprocessing) - (오류 수정 코드)
# ------------------------------------------------------------------------------
# AI가 학습할 수 있도록 데이터를 깨끗하게 다듬는 과정입니다.

print("\nStep 3: 데이터 전처리를 시작합니다.")

# 1. 불필요한 'Loan_ID' 컬럼 제거
#    errors='ignore'를 추가하여, 컬럼이 이미 삭제된 경우에도 오류 없이 넘어갑니다.
df = df.drop('Loan_ID', axis=1, errors='ignore') # <-- 여기가 수정되었습니다.

# 2. 결측치(NaN) 처리
#    - 범주형 컬럼은 최빈값(mode)으로 채움
cat_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])
#    - 수치형 컬럼은 중앙값(median)으로 채움
num_cols = ['LoanAmount', 'Loan_Amount_Term']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# 3. 범주형 변수 인코딩 (One-Hot Encoding)
#    (타겟 변수인 Loan_Status는 제외하고 피처들만 인코딩)
df_processed = pd.get_dummies(df, drop_first=True, columns=[
    'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'
])

# 4. 타겟 변수(Loan_Status)를 1(Y)과 0(N)으로 변환
df_processed['Loan_Status'] = df_processed['Loan_Status'].map({'Y': 1, 'N': 0})

# 5. 피처(X)와 타겟(y) 분리
X = df_processed.drop('Loan_Status', axis=1)
y = df_processed['Loan_Status']

print("  - 데이터 전처리가 완료되었습니다.")
print("\n  - 전처리된 피처(X) 데이터 미리보기 (앞 5개 컬럼):")
print(X.head(5).iloc[:, :5])


Step 3: 데이터 전처리를 시작합니다.
  - 데이터 전처리가 완료되었습니다.

  - 전처리된 피처(X) 데이터 미리보기 (앞 5개 컬럼):
   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0       128.0             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History  
0             1.0  
1             1.0  
2             1.0  
3             1.0  
4             1.0  


In [5]:
print(y.head())

0    1
1    0
2    1
3    1
4    1
Name: Loan_Status, dtype: int64


In [6]:
# ------------------------------------------------------------------------------
# 단계 4: 모델 학습 (Model Training)
# ------------------------------------------------------------------------------

# 전체 데이터를 학습용(80%)과 검증용(20%)으로 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nStep 4: 데이터를 학습용과 검증용으로 분리했습니다.")
print(f"  - 학습용 데이터 (X_train): {X_train.shape[0]} 건")
print(f"  - 검증용 데이터 (X_test) : {X_test.shape[0]} 건")


# '학습용' 데이터(X_train, y_train)를 사용하여 모델을 학습시킵니다.
print("\nStep 5: AI 모델 학습을 시작합니다.")

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("  - AI 모델 학습을 완료했습니다.")

# 학습에 사용되지 않은 '검증용' 데이터(X_test, y_test)로 모델의 성능을 공정하게 평가합니다.
print("\nStep 6: 검증용 데이터로 모델 성능을 평가합니다.")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"  - Accuracy (정확도): {accuracy:.4f}")

print("\n  - Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Refused (N)', 'Approved (Y)']))





Step 4: 데이터를 학습용과 검증용으로 분리했습니다.
  - 학습용 데이터 (X_train): 491 건
  - 검증용 데이터 (X_test) : 123 건

Step 5: AI 모델 학습을 시작합니다.
  - AI 모델 학습을 완료했습니다.

Step 6: 검증용 데이터로 모델 성능을 평가합니다.
  - Accuracy (정확도): 0.8374

  - Classification Report:
              precision    recall  f1-score   support

 Refused (N)       0.80      0.63      0.71        38
Approved (Y)       0.85      0.93      0.89        85

    accuracy                           0.84       123
   macro avg       0.82      0.78      0.80       123
weighted avg       0.83      0.84      0.83       123



In [7]:
# ------------------------------------------------------------------------------
# 단계 7: 새로운 가상 데이터로 예측 시뮬레이션 (Prediction Simulation)
# ------------------------------------------------------------------------------
# 모델이 새로운 데이터를 어떻게 예측하는지 시뮬레이션합니다.
print("\nStep 7: 새로운 가상 데이터로 예측을 시뮬레이션합니다.")

# 1. 가상의 대출 신청자 데이터 정의 (Raw Data)
sample_data = {
    'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate',
    'Self_Employed': 'No', 'ApplicantIncome': 5000, 'CoapplicantIncome': 2000,
    'LoanAmount': 150, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0,
    'Property_Area': 'Urban'
}
sample_df = pd.DataFrame([sample_data])
print("\n  [1. Raw Sample Data]")
print(sample_df)

# 2. Raw Data -> 전처리 적용 (One-Hot Encoding)
sample_df_processed = pd.get_dummies(sample_df)

# 3. 피처 정렬 (Feature Alignment) - ⭐ 매우 중요 ⭐
#    학습 시 사용된 'X.columns'를 기준으로 컬럼을 재정렬하고, 없는 컬럼은 0으로 채웁니다.
final_sample = sample_df_processed.reindex(columns=X.columns, fill_value=0)

print("\n  [2. Preprocessed Sample Data for Model (앞 5개 컬럼)]")
print(final_sample[final_sample.columns[:5]])

# 4. 최종 예측 수행
prediction = model.predict(final_sample)
prediction_proba = model.predict_proba(final_sample)

print("\n  [3. Prediction Result]")
if prediction[0] == 1:
    print(f"  -> 예측 결과: '승인 (Approved - Y)'")
else:
    print(f"  -> 예측 결과: '거절 (Refused - N)'")

print(f"  -> '거절(N)' 확률: {prediction_proba[0][0]:.2%}")
print(f"  -> '승인(Y)' 확률: {prediction_proba[0][1]:.2%}")


Step 7: 새로운 가상 데이터로 예측을 시뮬레이션합니다.

  [1. Raw Sample Data]
  Gender Married Dependents Education Self_Employed  ApplicantIncome  \
0   Male     Yes          1  Graduate            No             5000   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0               2000         150             360.0             1.0   

  Property_Area  
0         Urban  

  [2. Preprocessed Sample Data for Model (앞 5개 컬럼)]
   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5000               2000         150             360.0   

   Credit_History  
0             1.0  

  [3. Prediction Result]
  -> 예측 결과: '승인 (Approved - Y)'
  -> '거절(N)' 확률: 19.00%
  -> '승인(Y)' 확률: 81.00%


In [8]:
# ------------------------------------------------------------------------------
# 단계 8: 학습된 모델 저장하기 (Model Saving)
# ------------------------------------------------------------------------------
# 'models' 폴더를 만들어 학습된 모델과 피처 목록을 저장합니다.
print("\nStep 8: 학습된 모델을 파일로 저장합니다.")

MODEL_DIR = "models_debug2"
MODEL_DIR_ONLY = MODEL_DIR + '/model'
MODEL_FEATURE_DIR = MODEL_DIR+'/feat'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(MODEL_DIR_ONLY, exist_ok=True)
os.makedirs(MODEL_FEATURE_DIR, exist_ok=True)

MODEL_FILE_PATH = os.path.join(MODEL_DIR_ONLY, 'model.pkl')  # pkl 파일명 절대 수정하지 말것 
FEATURES_FILE_PATH = os.path.join(MODEL_FEATURE_DIR, 'model_features.pkl') # pkl 파일명 절대 수정하지 말것 

# 순수 pickle로 저장
with open(MODEL_FILE_PATH, "wb") as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

# 피처 목록도 동일하게 pickle로 저장
with open(FEATURES_FILE_PATH, "wb") as f:
    pickle.dump(list(X.columns), f, protocol=pickle.HIGHEST_PROTOCOL)



#joblib.dump(model, MODEL_FILE_PATH)
#joblib.dump(list(X.columns), FEATURES_FILE_PATH) # 예측 시 사용할 피처 순서 저장

print(f"  - 모델이 '{MODEL_FILE_PATH}' 경로에 성공적으로 저장되었습니다.")
print("  - JupyterLab 왼쪽 파일 탐색기에서 'models' 폴더와 그 안의 파일들을 확인하세요!")


Step 8: 학습된 모델을 파일로 저장합니다.
  - 모델이 'models_debug2/model/model.pkl' 경로에 성공적으로 저장되었습니다.
  - JupyterLab 왼쪽 파일 탐색기에서 'models' 폴더와 그 안의 파일들을 확인하세요!
